# Mobile Robotics Semester Project Notebook

**Course:** Mobile Robotics  
**Submitted to:** Dr. Talha Manzoor, Dr. AbuBakr Muhammad  
**Submitted by:** Zainab Ishtiaq  
**Roll #:** 24260003 MS Digital and Embedded Systems   
**Email:** 24260003 
**Date:** 23rd December 2025  

---

### Reference Paper
- **Paper title:** Camera-based mapping in search-and-rescue via flying andground robot teams  
- **Paper link:** https://doi.org/10.1007/s00138-024-01594-4
- **Video link:** https://youtu.be/Zv_d28XeWFc?si=4-9ERZR77axTCBW7
---

### Brief Summary of the Paper

The referenced paper proposes a camera-based victim detection, mapping, and tracking framework for Search and Rescue (SAR) using a team of aerial and ground robots. The paper exploit both aerial and ground robots (Fig 1) to provide distance and movement measures. Human victims are detected using pose-based DL model (YOLO8N), and missing or occluded body landmarks are reconstructed as well. Detected victim positions from multiple platforms are projected into a common ground plane and fused over time using a Kalman Filter (KF) to obtain smooth and robust 2D trajectories, even in the presence of noise and missed detection.

![Figure 1: Aerial–Ground SAR System](figs/fig_1.png)


In our project we have simulated the framework in real world using ultralytics models( yolo8n,10n) and focused on 2D tracking of the victim who is moving randomly. The video used is shot from the 2nd floor of the ground(aerial_view) and filtering is done for varying frame strides to showcase nature of a constant velocity KF filter. The sections are as follows:
####    1- Victim Detection
####    2- Victim Localization(2d)
####    3- Victim Tracking

# 1- Victim Detection
The paper models victim detection through **human pose estimation**, using a fixed set of **17 anatomical landmarks** (COCO keypoint format). These landmarks jointly define the posture, orientation, and spatial extent of a human body and are robust to partial occlusions compared to simple bounding boxes.

##### The 17 Body Landmarks


![Figure 2: 17 LM](figs/fig_2.png)

Reference: https://docs.ultralytics.com/tasks/pose/

These landmarks allow the system to reason about body orientation, limb symmetry, and approximate body dimensions, which are critical in SAR scenarios where victims may be partially visible.

### 1. Pose detection
Ultralytics YOLO requires setup of pytorch and some dependencies. Once model is downloaded our videos for Ground and Aerial views used as inputs.

Relevant output includes: boundingbox info, coorindates of victim, 17 landmarks, confidence


In [41]:
from ultralytics import YOLO
model = YOLO("yolov8n-pose.pt")

# process each video (ground and aerial)
results_ground = model("vid1_A_ground.mov", imgsz=640, conf=0.2)
results_aerial = model("Data/vid1_A_aerial.mp4", imgsz=640, conf=0.2)


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/494) c:\Users\LENOVO\Desktop\LUMS\mobile robotics\project\Semester_project_MR_24260003\Data\vid1_A_aerial.mp4: 384x640 1 person, 21.7ms
video 1/1 (frame 2/494) c:\Users\LENOVO\Desktop\LUMS\mobile robotics\project\Semester_project_MR_24260003\Data\vid1_A_aerial.mp4: 384x640 1 person, 17.2ms
video 1/1 (frame 3/494) c:\Users\LENOVO\Desktop\LUMS\mobile robotics\project\Semester_project_MR_24260003\Data\vid1_A_aerial.mp4: 384x640 1 person, 1

### 2. Visualize
Let's print the video with annotated 17 landmarks using cv2

In [42]:
import cv2

cap = cv2.VideoCapture("Data/vid1_A_aerial.mp4")

for frame_idx, r in enumerate(results_aerial):
    ret, frame = cap.read()
    if not ret:
        break

    if frame_idx % 10 != 0:
        continue

    clean = frame.copy()

    if r.keypoints is not None and len(r.keypoints.xy) > 0:
        kps = r.keypoints.xy[0]
        for (x, y) in kps:
            cv2.circle(clean, (int(x), int(y)), 2, (0,255,0), -1)

    small = cv2.resize(clean, (0,0), fx=1.5, fy=1.5)
    cv2.imshow("Aerial Video", small)

    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


![Figure 3: Detected pose](figs/fig_3.png)

### 3- Victim localization 
The paper estimates occluded position of victims based off symmetry and analytical measures. 

![Figure 4: Occlusions](figs/fig_4.png)

Instead of using the full body model as emphasized in the paper, I approximate the victim position using the **centroid of the detected keypoints**.

The centroid is computed as the mean of all valid (x, y) landmark coordinates:
- It is robust to missing limbs.
- It simplifies downstream processing.
- It provides a stable 2D position suitable for Kalman filtering.

This is a deliberate simplification that preserves the tracking objective while reducing geometric complexity.

In [43]:
def get_victim_xy(result, person_id=0, conf_thresh=0.3):
    """
    Compute victim (X, Y) position in image pixel coordinates
    using a confidence-weighted centroid of YOLOv8 pose keypoints.

    Returns
    -------
    (x, y) : tuple or None
        Victim position in pixel coordinates, or None if not enough data
    """

    # No pose detected or empty keypoints
    if result.keypoints is None or len(result.keypoints.data) == 0:
        return None
    
    # Check if person_id exists in the detected persons
    if person_id >= len(result.keypoints.data):
        return None

    # Extract keypoints for selected person
    # Shape: (17, 3) → [x, y, confidence]
    kps = result.keypoints.data[person_id]

    xy = kps[:, :2]      # (17, 2) pixel locations
    conf = kps[:, 2]     # (17,) confidence values

    # Keep only reliable keypoints
    valid = conf > conf_thresh
    if valid.sum() < 3:  # too few points → unstable centroid
        return None

    xy = xy[valid]
    conf = conf[valid]

    # Confidence-weighted centroid
    x = (xy[:, 0] * conf).sum() / conf.sum()
    y = (xy[:, 1] * conf).sum() / conf.sum()

    return float(x), float(y)

In [52]:
for r in results_aerial:
    victim_xy = get_victim_xy(r)
    
    if victim_xy is not None:
        x, y = victim_xy
        # Feed (x, y) into Kalman Filter
        print(f"Victim position: x={x:.1f}, y={y:.1f}")
    else:

        print("No victim detected in this frame")
        continue

Victim position: x=809.5, y=193.4
Victim position: x=808.7, y=194.7
Victim position: x=808.3, y=195.1
Victim position: x=807.7, y=196.4
Victim position: x=807.5, y=196.7
Victim position: x=807.5, y=197.1
Victim position: x=807.2, y=197.5
Victim position: x=807.1, y=197.8
Victim position: x=808.1, y=197.6
Victim position: x=806.2, y=199.4
Victim position: x=806.6, y=199.7
Victim position: x=806.1, y=200.4
Victim position: x=806.3, y=200.0
Victim position: x=806.2, y=200.1
Victim position: x=806.7, y=199.7
Victim position: x=807.8, y=200.2
Victim position: x=807.4, y=200.4
Victim position: x=807.1, y=201.0
Victim position: x=806.2, y=201.6
Victim position: x=807.3, y=200.8
Victim position: x=806.7, y=202.4
Victim position: x=807.1, y=202.9
Victim position: x=807.4, y=203.4
Victim position: x=807.5, y=203.8
Victim position: x=806.7, y=205.3
Victim position: x=808.4, y=203.6
Victim position: x=808.0, y=204.5
Victim position: x=807.2, y=205.3
Victim position: x=806.0, y=206.5
Victim positio

# 2- Victim Localization

localization is approximated using a **planar homography** approach:

1. Four reference points on the ground are selected.
2. Their real-world coordinates are manually measured in **meters**.
3. Corresponding pixel coordinates are extracted from the image.
4. A homography matrix \(H\) is computed using Direct Linear Transformation (DLT).
5. Detected victim centroids are projected from image space to world space.

This simulates the paper’s ground-plane projection step. While it does not include full robot pose estimation or SLAM, it is sufficient for demonstrating 2D tracking and sensor fusion.

By calculating the reference points in real world and converting those 4 points into pixels using homo_annotated.py i develop my Homography matrix

In [45]:
import numpy as np
import cv2

# Image points (pixels)
image_pts = np.array([
    [23, 375],    # bottom-left
    [740, 403],   # bottom-right
    [99, 93],     # top-left
    [733, 132]    # top-right
], dtype=np.float32)

# World points (meters) — rectangular ground plane
world_pts = np.array([
    [0.0, 5.5],     # bottom-left
    [11.5, 5.5],    # bottom-right
    [0.0, 0.0],     # top-left
    [11.5, 0.0]     # top-right
], dtype=np.float32)

H, _ = cv2.findHomography(image_pts, world_pts)


![Figure 5: Reference World to Pixel](figs/fig_5.png)

In [46]:
def pixel_to_world(xy_pixel, H):
    """
    Project image pixel coordinates to world coordinates (meters)
    using homography.
    """
    pt = np.array([[xy_pixel]], dtype=np.float32)
    world = cv2.perspectiveTransform(pt, H)
    return tuple(world[0, 0])


In [47]:
victim_track_world = []

for r in results_aerial:
    victim_px = get_victim_xy(r)

    if victim_px is None:
        continue

    victim_w = pixel_to_world(victim_px, H)
    victim_track_world.append(victim_w)

    print(f"Victim @ X = {victim_w[0]:.2f} m, Y = {victim_w[1]:.2f} m")


victim_track_world = np.array(victim_track_world)
print("\nVictim trajectory shape:", victim_track_world.shape)



Victim @ X = 12.90 m, Y = 1.27 m
Victim @ X = 12.89 m, Y = 1.30 m
Victim @ X = 12.88 m, Y = 1.31 m
Victim @ X = 12.87 m, Y = 1.34 m
Victim @ X = 12.86 m, Y = 1.34 m
Victim @ X = 12.86 m, Y = 1.35 m
Victim @ X = 12.86 m, Y = 1.36 m
Victim @ X = 12.85 m, Y = 1.37 m
Victim @ X = 12.87 m, Y = 1.36 m
Victim @ X = 12.84 m, Y = 1.40 m
Victim @ X = 12.84 m, Y = 1.41 m
Victim @ X = 12.83 m, Y = 1.43 m
Victim @ X = 12.84 m, Y = 1.42 m
Victim @ X = 12.84 m, Y = 1.42 m
Victim @ X = 12.84 m, Y = 1.41 m
Victim @ X = 12.86 m, Y = 1.42 m
Victim @ X = 12.86 m, Y = 1.42 m
Victim @ X = 12.85 m, Y = 1.44 m
Victim @ X = 12.83 m, Y = 1.45 m
Victim @ X = 12.86 m, Y = 1.43 m
Victim @ X = 12.84 m, Y = 1.47 m
Victim @ X = 12.85 m, Y = 1.48 m
Victim @ X = 12.85 m, Y = 1.49 m
Victim @ X = 12.86 m, Y = 1.50 m
Victim @ X = 12.84 m, Y = 1.53 m
Victim @ X = 12.87 m, Y = 1.49 m
Victim @ X = 12.86 m, Y = 1.51 m
Victim @ X = 12.85 m, Y = 1.53 m
Victim @ X = 12.82 m, Y = 1.56 m
Victim @ X = 12.85 m, Y = 1.52 m
Victim @ X

In [51]:
cap = cv2.VideoCapture("Data/vid1_A_aerial.mp4")

frame_idx = 200
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)  # <<< THIS WAS MISSING

ret, frame = cap.read()
cap.release()

if not ret:
    raise RuntimeError(f"Could not read frame {frame_idx}")

r = results_aerial[frame_idx]
victim_px = get_victim_xy(r)

if victim_px is None:
    raise RuntimeError("No victim detected in selected frame")

# Pixel → World
pt = np.array([[victim_px]], dtype=np.float32)
victim_world = cv2.perspectiveTransform(pt, H)[0, 0]

Xw, Yw = victim_world
x_px, y_px = int(victim_px[0]), int(victim_px[1])

origin_px = (99, 93)

cv2.circle(frame, origin_px, 8, (255, 0, 0), -1)
cv2.putText(frame, "World (0,0)",
            (origin_px[0] + 5, origin_px[1] - 5),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

cv2.circle(frame, (x_px, y_px), 8, (0, 0, 255), -1)
cv2.putText(frame, f"Victim ({Xw:.2f} m, {Yw:.2f} m)",
            (x_px + 5, y_px - 5),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

cv2.imwrite("victim_localization_frame200.png", frame)
cv2.imshow("Victim Localization (World Coordinates)", frame)
cv2.waitKey(10000)
cv2.destroyAllWindows()


![Figure 6: Localized 2D](figs/fig_6.png)

# 3- Victim Tracking

## Kalman Filter (KF)

### State Definition and Motion Model

We adopt a **2D constant-velocity motion model**, consistent with the formulation used in the paper:

\begin{equation}
\mathbf{x}_k =
\begin{bmatrix}
p_x(k) \\
p_y(k) \\
v_x(k) \\
v_y(k)
\end{bmatrix}
\end{equation}

where \(p_x, p_y\) denote the target position in pixel coordinates and  
\(v_x, v_y\) denote the corresponding velocity components.

The discrete-time state transition model is given by:

\begin{equation}
\mathbf{x}_{k|k-1} = \mathbf{A}\mathbf{x}_{k-1|k-1} + \mathbf{w}_k
\end{equation}

with

\begin{equation}
\mathbf{A} =
\begin{bmatrix}
1 & 0 & \Delta t & 0 \\
0 & 1 & 0 & \Delta t \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
\end{equation}

and process noise

\begin{equation}
\mathbf{w}_k \sim \mathcal{N}(0, \mathbf{Q})
\end{equation}

This model assumes linear motion with approximately constant velocity between consecutive video frames, which is a reasonable assumption for human walking motion at typical frame rates.

---

### Measurement Model

Only 2D position measurements are available from pose detection. The measurement model is therefore:

\begin{equation}
\mathbf{z}_k = \mathbf{H}\mathbf{x}_k + \mathbf{v}_k
\end{equation}

where

\begin{equation}
\mathbf{H} =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0
\end{bmatrix}
\end{equation}

and the measurement noise is modeled as:

\begin{equation}
\mathbf{v}_k \sim \mathcal{N}(0, \mathbf{R})
\end{equation}

No direct velocity measurements are available. Velocity is treated as a **hidden state** and is implicitly estimated by the Kalman Filter through successive position updates.

---

### Kalman Filter Equations

**Prediction step:**

\begin{equation}
\hat{\mathbf{x}}_{k|k-1} = \mathbf{A}\hat{\mathbf{x}}_{k-1|k-1}
\end{equation}

\begin{equation}
\mathbf{P}_{k|k-1} = \mathbf{A}\mathbf{P}_{k-1|k-1}\mathbf{A}^\top + \mathbf{Q}
\end{equation}

**Innovation (measurement residual):**

\begin{equation}
\mathbf{y}_k = \mathbf{z}_k - \mathbf{H}\hat{\mathbf{x}}_{k|k-1}
\end{equation}

\begin{equation}
\mathbf{S}_k = \mathbf{H}\mathbf{P}_{k|k-1}\mathbf{H}^\top + \mathbf{R}
\end{equation}

**Kalman gain:**

\begin{equation}
\mathbf{K}_k = \mathbf{P}_{k|k-1}\mathbf{H}^\top \mathbf{S}_k^{-1}
\end{equation}

**Update step:**

\begin{equation}
\hat{\mathbf{x}}_{k|k} = \hat{\mathbf{x}}_{k|k-1} + \mathbf{K}_k \mathbf{y}_k
\end{equation}

\begin{equation}
\mathbf{P}_{k|k} = (\mathbf{I} - \mathbf{K}_k \mathbf{H})\mathbf{P}_{k|k-1}
\end{equation}

---

### Noise Modeling and Tuning

- **Process noise covariance Q** accounts for unmodeled acceleration and motion uncertainty.
- **Measurement noise covariance R** captures pose-detection noise in pixel space.

The tuning follows the same principle described in the paper:  
the filter relies more on the motion model when measurements are unreliable and more on measurements when detections are consistent.

## 1- Setup


In [ ]:
class PixelKalmanFilter:
    def __init__(self, dt=1/30):
        # State vector: x_k = [px, py, vx, vy]^T
        self.x = np.zeros((4,1))              # \hat{x}_{k|k}
        self.P = np.eye(4) * 100.0            # P_{k|k}

        self.dt = dt

        # State transition matrix A
        self.A = np.array([
            [1, 0, dt, 0],
            [0, 1, 0, dt],
            [0, 0, 1,  0],
            [0, 0, 0,  1]
        ])

        # Measurement matrix H
        self.H = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])

        # TUNED FROM 0.01, 25
        # Process noise covariance Q
        self.Q = np.eye(4) * 3

        # Measurement noise covariance R
        self.R = np.eye(2) * 20

        self.I = np.eye(4)

    def predict(self):
        # Prediction:
        # x_{k|k-1} = A x_{k-1|k-1}
        # P_{k|k-1} = A P_{k-1|k-1} A^T + Q
        self.x = self.A @ self.x
        self.P = self.A @ self.P @ self.A.T + self.Q
        return self.x

    def update(self, z):
        if z is None:
            return

        z = np.array(z).reshape((2,1))  # z_k

        # Innovation (residual)
        y = z - self.H @ self.x         # y_k

        # Innovation covariance
        S = self.H @ self.P @ self.H.T + self.R  # S_k

        # Kalman gain
        K = self.P @ self.H.T @ np.linalg.inv(S) # K_k

        # State update
        self.x = self.x + K @ y         # x_{k|k}

        # Covariance update
        self.P = (self.I - K @ self.H) @ self.P  # P_{k|k}

    def state(self):
        return self.x.flatten()


## 2- Filtering
 
Here we will track the predictions(P), update(U) and measurements(z). 
We will also vary states by varying number of frames that the filter takes in


In [61]:
cap = cv2.VideoCapture("Data/vid1_A_aerial.mp4")
kf = PixelKalmanFilter(dt=1/30)

frame_idx = 0
frame_stride = 5  # visualize every 20th frame

pred_history = []
meas_history = []
upd_history  = []

# --- Warm-up settings ---
kf_steps = 0
WARMUP_STEPS = 3   # ignore first 5 KF updates


while True:
    ret, frame = cap.read()
    if not ret or frame_idx >= len(results_aerial):
        break

    # --- VISUALIZE EVERY 20th FRAME ---
    if frame_idx % frame_stride == 0:
        r = results_aerial[frame_idx]

        # --- KF PREDICTION ---
        x_pred = kf.predict()
        pred_pt = (int(x_pred[0]), int(x_pred[1]))

        # --- MEASUREMENT ---
        victim_xy = get_victim_xy(r)

        # --- KF UPDATE ---
        kf.update(victim_xy)
        kf_steps += 1

        x_est, y_est, vx_est, vy_est = kf.state()
        upd_pt = (int(x_est), int(y_est))

        # ------------------ STORE TRACKING PATH (after warm-up) ------------------
        if kf_steps > WARMUP_STEPS:
            pred_history.append(pred_pt)
            upd_history.append(upd_pt)

            if victim_xy is not None:
                meas_pt = (int(victim_xy[0]), int(victim_xy[1]))
                meas_history.append(meas_pt)

        # ------------------ DRAW MEASUREMENT ------------------
        if victim_xy is not None:
            meas_pt = (int(victim_xy[0]), int(victim_xy[1]))
            cv2.circle(frame, meas_pt, 6, (0, 0, 255), -1)
            cv2.putText(frame, "Measurement", (meas_pt[0]+5, meas_pt[1]),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)

        # ------------------ DRAW POINTS ------------------
        cv2.circle(frame, pred_pt, 6, (255, 0, 0), -1)
        cv2.putText(frame, "Prediction", (pred_pt[0]+5, pred_pt[1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)

        cv2.circle(frame, upd_pt, 6, (0, 255, 0), -1)
        cv2.putText(frame, "Update", (upd_pt[0]+5, upd_pt[1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

        # ------------------ DRAW TRACKING PATHS ------------------
        for i in range(1, len(pred_history)):
            cv2.line(frame, pred_history[i-1], pred_history[i], (255, 0, 0), 2)

        for i in range(1, len(upd_history)):
            cv2.line(frame, upd_history[i-1], upd_history[i], (0, 255, 0), 2)

        for i in range(1, len(meas_history)):
            cv2.line(frame, meas_history[i-1], meas_history[i], (0, 0, 255), 2)

        cv2.imshow("KF Victim Tracking (Aerial)", frame)

        if cv2.waitKey(200) & 0xFF == ord('q'):
            break

    frame_idx += 1

cap.release()
cv2.destroyAllWindows()


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9240\3963850996.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_pt = (int(x_pred[0]), int(x_pred[1]))


For every 5 frames:

![Figure 7: KF visualized](figs/fig_7.png)

for every 20 frames:

![Figure 8: KF visualizedd](figs/fig_8.png)

## Conclusion

This notebook demonstrates a simplified yet faithful replication of the paper’s detection, localization, and tracking pipeline. While several components are approximated (centroid-based victim localization, planar homography, 2D motion model), the core ideas—robust human detection, projection into world coordinates, and Kalman-filter-based trajectory estimation—are preserved and validated through implementation.

For elevation estmation, following data has been recorded that can be used for distance edtimation and Z location of the victim.

# 4-Future recommendations
------------------------------
CAMERA & SCENE SPECIFICATIONS
------------------------------

Victim real height (S_real):
- 63 inches
- 1.60 meters

Victim image height (S_img):
- 66 pixels

------------------------------
AERIAL CAMERA (Samsung A25)
------------------------------
Camera height (Z):
- 9.5 meters

Camera tilt:
- 45 degrees downward

Camera horizontal FOV:
- 76 degrees

Camera vertical FOV:
- 56 degrees

Fig 7 from paper:

![Figure 9: FOVs](figs/fig_9.png)

Camera position relative to world origin (0,0):
- X_cam = 4.5 meters
- Y_cam = 8.8 meters

Ground reference dimensions:
- Width (X): 11.5 meters
- Height (Y): 5.5 meters

Homography reference:
World (meters):
(0.0, 0.0)     -> top-left
(11.5, 0.0)    -> top-right
(0.0, 5.5)     -> bottom-left
(11.5, 5.5)    -> bottom-right

Image (pixels):
top-left     = (99, 93)
top-right    = (733, 132)
bottom-left  = (23, 375)
bottom-right = (740, 403)

------------------------------
GROUND CAMERA (iPhone 11)
------------------------------
Camera height (Z):
- 2.75 meters

Camera horizontal FOV:
- 77 degrees

Camera vertical FOV:
- 59 degrees

Camera position relative to world origin (0,0):
- X_cam = 13.0 meters
- Y_cam = 0.0 meters

Reference pixel corresponding to world (0,0):
- (93, 99)

![Figure 10: Drawing](figs/fig_10.jpg)
------------------------------
IMAGE PARAMETERS
------------------------------
Image width (W_img):
- from code

Image height (H_img):
- from code
